In [127]:
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import LabelBinarizer

In [136]:
train = np.loadtxt('iris/iris-train.txt')
test = np.loadtxt('iris/iris-test.txt')
XTrain = np.append(train[:,1:],np.ones((train.shape[0],1)),axis=1)
yTrain = train[:,0]
TTrain = LabelBinarizer().fit_transform(yTrain).T
XTest = np.append(test[:,1:],np.ones((test.shape[0],1)),axis=1)
yTest = test[:,0]
TTest = LabelBinarizer().fit_transform(yTest).T
print("XTrain:",XTrain.shape)
print("XTest:",XTest.shape)
print("TTrain:",TTrain.shape)
print("TTest:",TTest.shape)

XTrain: (90, 3)
XTest: (51, 3)
TTrain: (3, 90)
TTest: (3, 51)


In [129]:
# def musicMSE(pred, gt):
#     mse = (np.square(np.rint(pred)-gt)).mean()
#     return mse
# print(musicMSE(np.zeros(3),np.ones(3)))

In [137]:
def P(X,W): #will be kxn (3 classes by n examples)
    mat = np.matmul(W.T,X)
    num = np.exp(mat-np.max(mat))
    return num/np.sum(num,axis=0)

In [150]:
def gradient_descent(XTrain,TTrain,XTest,TTest,alpha,lr,epochs=100,momentum=0,batch_size = 128):
    W = np.zeros((TTrain.shape[0],XTrain.shape[1]))
    dW = np.zeros((TTrain.shape[0],XTrain.shape[1]))
    nTrain, featCount = XTrain.shape
    nTest = XTest.shape[0]

    def P(X,W): #will be nxk (3 classes by n examples)
        mat = np.matmul(X,W)
        num = np.exp(np.exp(mat-np.max(mat)))
        return num/np.sum(num,axis=0) 
    def getgrad(X,T,W):
        return alpha*W + (np.matmul((P(X,W).T-T),X))
    def getlosses(XTrain,XTest,TTrain,TTest,W):
        train_loss = -np.sum(np.matmul(TTrain,np.log(P(XTrain,W))))/nTrain
        test_loss = -np.sum(np.matmul(TTest,np.log(P(XTest,W))))/nTest
        return train_loss,test_loss
    def getaccuracies(X,T,W):
        guesses = np.argmax(P(X,W),axis=1)
        truths = np.argmax(T,axis=0)
        acc = 0
        for i in range(len(guesses)):
            if guesses[i] == truths[i]:
                acc+=1
        return acc/len(guesses)
    XTrain2 = np.copy(XTrain)
    XTrain2[:,:-1] = 2*(XTrain2[:,:-1]-0.5)
    XTest2 = np.copy(XTest)
    XTest2[:,:-1] = 2*(XTest2[:,:-1]-0.5)
    TTrain2 = np.copy(TTrain)
    TTest2 = np.copy(TTest)
    test_losses = []
    train_losses = []
    test_accuracies = []
    train_accuracies = []
    for i in range(epochs):
        #shuffle train data:
        p = random.permutation(XTrain2.shape[0])
        XTrain2 = XTrain2[p]
        TTrain2 = TTrain2[:,p]
        for b in range(math.ceil(XTrain2.shape[0]/batch_size)):
            if(batch_size*(b+1) < XTrain2.shape[0]):
                batchXTrain = np.copy(XTrain2[batch_size*b:batch_size*(b+1)])
                batchTTrain = np.copy(TTrain2[:,batch_size*b:batch_size*(b+1)])
            else:
                batchXTrain = np.copy(XTrain2[batch_size*b:-1])
                batchTTrain = np.copy(TTrain2[:,batch_size*b:-1])
            grad = getgrad(batchXTrain,batchTTrain,W)
            # print("train:",batchTTrain[:,:5])
            # print("Grad:",grad)
            dW = lr*dW + grad
            # print("dW:",dW)
            W -= dW
            # print("W:",W)
            # print(W)
        train_loss,test_loss = getlosses(XTrain2,XTest2,TTrain2,TTest2,W)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(getaccuracies(XTrain2,TTrain2,W))
        test_accuracies.append(getaccuracies(XTest2,TTest2,W))
        print("Epoch",i,"- Train Loss:",train_loss,"\t Test Loss: ",test_loss)

    return(W,train_losses,test_losses,train_accuracies,test_accuracies)
    # return(w,train_losses,test_losses)



In [ ]:
softMax = gradient_descent(XTrain,TTrain,XTest,TTest,
                         alpha = .0001,
                         lr = .0001,
                         epochs = 40,
                         batch_size = 8)

In [ ]:
# losses = np.array(losses)
start = 0
stop = 40
# plt.plot(range(start,stop),softMax[1][start:stop])
plt.plot(range(start,stop),softMax[2][start:stop])
plt.legend(["test"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("SoftMax, SGD w/ minibatch")
plt.show()
plt.plot(range(start,stop),softMax[3][start:stop])
plt.plot(range(start,stop),softMax[4][start:stop])
plt.legend(["train","test"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("SoftMax, SGD w/ minibatch")
plt.show()
print(softMax[0].shape)
print(softMax[0])
# print(P(XTest,softMax[0]))


In [ ]:
count = gradient_descent(trainFeat,trainYears,testFeat,testYears,
                         alpha = .0005,
                         lr = .0005,
                         type = "count",
                         epochs = 40,
                         batch_size = 1024)

In [ ]:
print(musicMSE(predict(testFeat,ridge),testYears))
print(musicMSE(predict(testFeat,ridge),testYears))
print(musicMSE(predict(testFeat,lasso),testYears))
print(musicMSE(predict(testFeat,count),testYears))

# def errorVec(pred, gt):
#     mse = (np.square(np.rint(pred)-gt))
#     return mse
# print(errorVec(testYears,predict(testFeat,ridge)))